<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br /><br>M. Sc. Carsten Bauer</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Vorlesung &mdash; Programmiertechniken 5</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2019</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2019-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2019-CompPhys.shtml)

In [ ]:
using DifferentialEquations, PyPlot

Links: [DifferentialEquations.jl](https://github.com/JuliaDiffEq/DifferentialEquations.jl) ([Dokumentation](https://docs.juliadiffeq.org/latest/index.html))

# Das N-Körper-Problem in 2 Dimensionen

Wir untersuchen ein System aus $N$ Massen, die sich gravitativ nach dem Gesetz
\begin{equation}
F_{ij} = -Gm_im_j\dfrac{\vec{r}_i - \vec{r}_j}{r^3}
\end{equation}
anziehen.
Die Gravitationskonstante sowie alle Massen setzen wir auf $1$.

In [ ]:
function flow(u, p, t)
    N = div(length(u), 4)
    return_vec = zeros(4 * N)
    
    # Zerlege den Vektor
    xs = u[1:N]
    ys = u[N+1 : 2*N]
    
    v_xs = u[2 * N + 1:3 * N]        
    v_ys = u[3 * N + 1:4 * N]
    return_vec[1:N] = v_xs
    return_vec[N + 1:2*N] = v_ys
    
    for i in 1:N
        for j in 1:N
            if i == j continue end
            r = sqrt((xs[i] - xs[j])^2 + (ys[i] - ys[j])^2)
            return_vec[2 * N + i] += - (xs[i] - xs[j]) / r^3
            return_vec[3 * N + i] += - (ys[i] - ys[j]) / r^3
        end
    end

    return return_vec
end

## 3-Körper

Anfangsbedingungen:

\begin{aligned}
\vec{x}_1 &= (0.97000436, -0.24308753) \\
\vec{x}_2 &= - \vec{x}_1  \\
\vec{x}_3 &= (0, 0) 
\end{aligned}

\begin{aligned}
\vec{v}_1 &= ( 0.466203685, 0.43236573 )\\
\vec{v}_2 &= \vec{v}_1\\
\vec{v}_3 &= -2\vec{v}_1
\end{aligned}

In [ ]:
ts = (1.0, 100)

initial_xs = [0.97000436, -0.97000436, 0.]
initial_ys = [-0.24307853, 0.24307853, 0.]
initial_v_xs = [0.466203685, 0.466203685, -2 * 0.466203685]
initial_v_ys = [0.43236573, 0.43236573, -2 * 0.43236573]
initial = [initial_xs; initial_ys; initial_v_xs; initial_v_ys]

prob = ODEProblem(flow, initial, ts)
sol = solve(prob, saveat=0.005);

In [ ]:
# Visualisierung
x1s = getindex.(sol.u, 1)
x2s = getindex.(sol.u, 2)
x3s = getindex.(sol.u, 3)
y1s = getindex.(sol.u, 4)
y2s = getindex.(sol.u, 5)
y3s = getindex.(sol.u, 6);


fig = figure(facecolor="white")
plot(x1s, y1s, color="C0", alpha=.4, linewidth=0.8)
plot(x2s, y2s, color="C1", alpha=.4, linewidth=0.8)
plot(x3s, y3s, color="C2", alpha=.4, linewidth=0.8)

p1 = plot(x1s[1], y1s[1], "o", markeredgecolor="none", markersize=20)[1]
p2 = plot(x2s[1], y2s[1], "o", markeredgecolor="none", markersize=20)[1]
p3 = plot(x3s[1], y3s[1], "o", markeredgecolor="none", markersize=20)[1]

xlim([minimum([x1s; x2s; x3s]) * 1.1, maximum([x1s; x2s; x3s]) * 1.1])
ylim([minimum([y1s; y2s; y3s]) * 1.1, maximum([y1s; y2s; y3s]) * 1.1])
axis("off")

In [ ]:
# Animation
using PyPlot, PyCall, ProgressMeter

# general animation creation
function generateAnimation(fig, update, frames, fps)
    anim = pyimport("matplotlib.animation")
    HTML = pyimport("IPython.display").HTML

    # init ProgressMeter
    progressbar = Progress(frames, 1, "Generating animation ... ")

    # animate function for creating frame i
    function animate(i)
        update(i)
        ProgressMeter.update!(progressbar, i+1)
        return (Union{})
    end

    # create animation object
    animation = anim.FuncAnimation(fig, animate, frames=frames, interval=fps)
    # animation.save(filename_output)
    # close(gcf())
    HTML(animation.to_html5_video())
end

up(i) = begin
    i = i+1
    p1.set_data(x1s[i], y1s[i])
    p2.set_data(x2s[i], y2s[i])
    p3.set_data(x3s[i], y3s[i])
end;

generateAnimation(fig, up, 1000, 2)

Wie wir sehen können, ist das System stabil.

Die mathematisch Interessierten finden die Herleitung dieses stabilen $3$-Körper-Systems in [diesem Artikel](http://arxiv.org/pdf/math/0011268.pdf) aus dem Jahre 2000.

### 4 Körper

In [ ]:
ts = (1.0, 100)

initial_xs = [1.382857, 0., -1.382857, 0.]
initial_ys = [0., 0.157030, 0., -0.157030]
initial_v_xs = [0, 1.871935, 0, -1.871935]
initial_v_ys = [0.584873, 0., -0.584873, 0.]
initial = [initial_xs; initial_ys; initial_v_xs; initial_v_ys]

prob = ODEProblem(flow, initial, ts)
sol = solve(prob, saveat=0.001);

In [ ]:
# Visualisierung
x1s = getindex.(sol.u, 1)
x2s = getindex.(sol.u, 2)
x3s = getindex.(sol.u, 3)
x4s = getindex.(sol.u, 4)
y1s = getindex.(sol.u, 5)
y2s = getindex.(sol.u, 6)
y3s = getindex.(sol.u, 7);
y4s = getindex.(sol.u, 8);

fig = figure(facecolor="white")
plot(x1s[1:10000], y1s[1:10000], color="C0", alpha=.4, linewidth=0.8)
plot(x2s[1:10000], y2s[1:10000], color="C1", alpha=.4, linewidth=0.8)
plot(x3s[1:10000], y3s[1:10000], color="C2", alpha=.4, linewidth=0.8)
plot(x4s[1:10000], y4s[1:10000], color="C3", alpha=.4, linewidth=0.8)

p1 = plot(x1s[1], y1s[1], "o", markeredgecolor="none", markersize=20)[1]
p2 = plot(x2s[1], y2s[1], "o", markeredgecolor="none", markersize=20)[1]
p3 = plot(x3s[1], y3s[1], "o", markeredgecolor="none", markersize=20)[1]
p4 = plot(x4s[1], y4s[1], "o", markeredgecolor="none", markersize=20)[1]

# xlim([minimum([x1s; x2s; x3s; x4s]) * 1.1, maximum([x1s; x2s; x3s; x4s]) * 1.1])
# ylim([minimum([y1s; y2s; y3s; y4s]) * 1.1, maximum([y1s; y2s; y3s; y4s]) * 1.1])
axis("off");

In [ ]:
# Animation
up(i) = begin
    i = i+1
    p1.set_data(x1s[i], y1s[i])
    p2.set_data(x2s[i], y2s[i])
    p3.set_data(x3s[i], y3s[i])
    p3.set_data(x4s[i], y4s[i])
end;

generateAnimation(fig, up, 10000, 1)

Wie wir sehen ist das System numerisch instabil. Kleine Fehler in der Integration wirken sich schnell stark aus.